In [7]:
import pandas as pd
import math
import statsmodels.api as sm
import numpy as np

In [139]:
df = pd.read_excel('base_1D.xlsx', index_col='Data')

In [140]:
def calc_ret_ln(df):
    #Cria matriz para receber valores calculados
    df_ln = pd.DataFrame(columns = df.columns, index = df.index)

    #Calcula o ln para todos os valores
    for i in range(len(df)):
        for columns in df.columns:
            df_ln[columns][i] = math.log(df[columns][i])

    # Retorno diário     
    ret = df_ln - df_ln.shift(-1)

    return ret.dropna(), df_ln

In [141]:
retorno, df_ln = calc_ret_ln(df)

In [142]:
def reg_m(y, x):
    ones = np.ones(len(x))
    X = sm.add_constant(np.column_stack((x, ones)))
    for ele in x:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results

In [143]:
periodo = 60

In [144]:
y = np.array(retorno.WINFUT.iloc[:periodo].values, dtype = float)
x = np.array(retorno.iloc[:periodo,1:].values, dtype = float)

In [145]:
def reg_m(x,y):
    X = sm.add_constant(x)
    modelo = sm.OLS(y, X)
    modelo_v1 = modelo.fit()
    return modelo_v1

In [146]:
print(reg_m(x,y).summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     30.78
Date:                Wed, 06 Oct 2021   Prob (F-statistic):           8.60e-10
Time:                        00:26:57   Log-Likelihood:                 193.82
No. Observations:                  60   AIC:                            -381.6
Df Residuals:                      57   BIC:                            -375.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.001     -1.612      0.1

In [147]:
import plotly.express as px

fig = plt.figure(figsize=(30,30))
fig = px.scatter_3d(retorno, x='WDOFUT', y='WSPFUT', z='WINFUT')
fig.show()

<Figure size 2160x2160 with 0 Axes>

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 100)

In [149]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [150]:
y_pred = regressor.predict(X_test)
print(y_pred)

[-0.01692723  0.00339457  0.00202496  0.00924415  0.00246989 -0.01286407
 -0.01758848 -0.00205849 -0.00336157 -0.02084911  0.00343182 -0.01107105
 -0.00408215  0.00075538  0.00197929]


In [151]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.5458219186848452